# Machine Learning-Based Forecasting of Bitcoin Volatility

## Data Cleaning and Preprocessing for Bitcoin Data

In [101]:
import yfinance as yf
import pandas as pd

btc = yf.download(
    "BTC-USD",
    start="2023-01-01",  
    auto_adjust=True
)

print(btc.head())

btc.to_csv("data/raw/bitcoin_daily.csv")

[*********************100%***********************]  1 of 1 completed

Price              Close          High           Low          Open  \
Ticker           BTC-USD       BTC-USD       BTC-USD       BTC-USD   
Date                                                                 
2023-01-01  16625.080078  16630.439453  16521.234375  16547.914062   
2023-01-02  16688.470703  16759.343750  16572.228516  16625.509766   
2023-01-03  16679.857422  16760.447266  16622.371094  16688.847656   
2023-01-04  16863.238281  16964.585938  16667.763672  16680.205078   
2023-01-05  16836.736328  16884.021484  16790.283203  16863.472656   

Price            Volume  
Ticker          BTC-USD  
Date                     
2023-01-01   9244361700  
2023-01-02  12097775227  
2023-01-03  13903079207  
2023-01-04  18421743322  
2023-01-05  13692758566  


In [102]:
btc = btc[['Close', 'Volume', 'High', 'Low']]
btc = btc.dropna()
btc.head()

Price,Close,Volume,High,Low
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,
2023-01-01,16625.080078,9244361700,16630.439453,16521.234375
2023-01-02,16688.470703,12097775227,16759.343750,16572.228516
2023-01-03,16679.857422,13903079207,16760.447266,16622.371094
2023-01-04,16863.238281,18421743322,16964.585938,16667.763672
2023-01-05,16836.736328,13692758566,16884.021484,16790.283203


In [103]:
import numpy as np

btc['log_return'] = np.log(btc['Close'] / btc['Close'].shift(1))
btc = btc.dropna()
btc.head()

# 20-day future volatility (target variable)
btc['target_vol_5d'] = (
    btc['log_return']
    .rolling(window=5)
    .std()
    .shift(-5)  
)

btc = btc.dropna()
btc.head()

Price,Close,Volume,High,Low,log_return,target_vol_5d
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,,
Date,,,,,,
2023-01-02,16688.470703,12097775227,16759.343750,16572.228516,0.003806,0.005445
2023-01-03,16679.857422,13903079207,16760.447266,16622.371094,-0.000516,0.005334
2023-01-04,16863.238281,18421743322,16964.585938,16667.763672,0.010934,0.004304
2023-01-05,16836.736328,13692758566,16884.021484,16790.283203,-0.001573,0.005079
2023-01-06,16951.968750,14413662913,16991.994141,16716.421875,0.006821,0.010454


### Feature Engineering

In [104]:
# Past volatility
btc['vol_5d'] = btc['log_return'].rolling(5).std()
btc['vol_20d'] = btc['log_return'].rolling(20).std()
btc['vol_30d'] = btc['log_return'].rolling(30).std()
btc['close'] = btc['Close'].shift(1)

# Momentum
btc['return_5d'] = btc['log_return'].rolling(5).sum()

# Volume change
btc['vol_change'] = btc['Volume'].pct_change()

btc = btc.dropna()


btc.to_csv("data/processed/bitcoin_daily_processed.csv")

btc.head()



Price,Close,Volume,High,Low,log_return,target_vol_5d,vol_5d,vol_20d,vol_30d,close,return_5d,vol_change
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,,,,,,,,
Date,,,,,,,,,,,,
2023-01-31,23139.283203,22837828665,23225.021484,22765.568359,0.013012,0.015921,0.026413,0.027947,0.023266,22840.138672,0.004613,-0.160547
2023-02-01,23723.769531,26683255504,23764.539062,22877.750000,0.024946,0.005779,0.028553,0.026662,0.023360,23139.283203,0.027566,0.168380
2023-02-02,23471.871094,32066936882,24167.210938,23468.595703,-0.010675,0.014386,0.029372,0.025140,0.023616,23723.769531,0.018958,0.201763
2023-02-03,23449.322266,27083066007,23678.103516,23279.955078,-0.000961,0.015361,0.024881,0.022963,0.023723,23471.871094,-0.013775,-0.155421
2023-02-04,23331.847656,15639298538,23556.949219,23291.794922,-0.005022,0.025611,0.014500,0.022974,0.023794,23449.322266,0.021300,-0.422543


## Training

In [105]:
split = int(len(btc) * 0.8)

train = btc.iloc[:split]
test = btc.iloc[split:]

features = [
    'vol_5d', 'vol_20d', 'vol_30d',
    'return_5d', 'vol_change', 'close'
]

X_train = train[features]
y_train = train['target_vol_5d']

X_test = test[features]
y_test = test['target_vol_5d']

In [106]:
from sklearn.metrics import mean_squared_error

baseline = X_test['vol_5d']
rmse = np.sqrt(mean_squared_error(y_test, baseline))
print("Baseline RMSE:", rmse)

Baseline RMSE: 0.016719560405765988


### XGBoost Model

In [107]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_squared_error

tscv = TimeSeriesSplit(n_splits=5)

xgb = XGBRegressor(
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)

param_dist = {
    "n_estimators": [200, 400, 800, 1200],
    "learning_rate": [0.01, 0.03, 0.05, 0.1],
    "max_depth": [2, 3, 4, 5],
    "min_child_weight": [1, 3, 5, 10],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "gamma": [0, 0.1, 0.3, 1.0],
    "reg_alpha": [0, 1e-4, 1e-3, 1e-2, 0.1],
    "reg_lambda": [0.5, 1.0, 2.0, 5.0],
}

search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=60,
    scoring="neg_root_mean_squared_error",
    cv=tscv,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

search.fit(X_train, y_train)

best_model = search.best_estimator_
xg_preds = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, xg_preds))
print("XGBoost RMSE:", rmse)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
XGBoost RMSE: 0.01273088279231121


### GARCH Model

In [108]:
from arch import arch_model
from sklearn.metrics import mean_squared_error
import numpy as np

train_returns = train['log_return'] * 100
test_returns  = test['log_return'] * 100

garch = arch_model(
    train_returns,
    mean='Zero',
    vol='GARCH',
    p=1,
    q=1
)

garch_fit = garch.fit(disp='off')

garch_forecasts = []

for i in range(len(test_returns)):
    rolling_train = returns = (
        btc['log_return'][:split + i] * 100
    )
    
    model = arch_model(
        rolling_train,
        mean='Zero',
        vol='GARCH',
        p=1,
        q=1
    )
    
    res = model.fit(disp='off')
    forecast = res.forecast(horizon=5)
    variance = forecast.variance.values[-1, 4]
    vol = np.sqrt(variance) / 100  
    garch_forecasts.append(vol)

garch_forecasts = np.array(garch_forecasts)

rmse_garch = np.sqrt(mean_squared_error(y_test[:len(garch_forecasts)], garch_forecasts))

print("GARCH RMSE:", rmse_garch)

GARCH RMSE: 0.014567245555306037


### HAR Model

In [109]:
from sklearn.linear_model import LinearRegression

har = LinearRegression()
har.fit(X_train[['vol_5d','vol_20d','vol_30d']], y_train)

har_preds = har.predict(X_test[['vol_5d','vol_20d','vol_30d']])

rmse_har = np.sqrt(mean_squared_error(y_test, har_preds))
print("HAR RMSE:", rmse_har)

HAR RMSE: 0.01347436458383007


## Model Evaluation

In [110]:
import numpy as np
from scipy import stats


e_baseline = y_test.values - baseline.values
e_xgb      = y_test.values - xg_preds

loss_baseline = e_baseline**2
loss_xgb      = e_xgb**2

d = loss_baseline - loss_xgb


mean_d = np.mean(d)
var_d  = np.var(d, ddof=1)

dm_stat = mean_d / np.sqrt(var_d / len(d))

# Two-sided p-value
p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))

print("Diebold-Mariano Test")
print("DM Statistic:", dm_stat)
print("p-value:", p_value)

Diebold-Mariano Test
DM Statistic: 2.2533816972721015
p-value: 0.024235092545610737


With a significance level of 0.05, XGBoost significantly outperforms the naive volatility forecast.

In [111]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

last_week_actual = btc["target_vol_5d"].iloc[-7:]
last_x = btc[features].iloc[[-1]]
next_vol_pred = float(best_model.predict(last_x)[0])
future_dates = pd.bdate_range(btc.index[-1] + pd.Timedelta(days=1), periods=5)
future_forecast = pd.Series([next_vol_pred] * 5, index=future_dates, name="Forecast")

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=last_week_actual.index,
    y=last_week_actual.values,
    mode="lines+markers",
    name="Realized (Last 7 Days)"
))

fig.add_trace(go.Scatter(
    x=future_forecast.index,
    y=future_forecast.values,
    mode="lines",
    line=dict(dash="dash"),
    name="Forecast (Next 5 Days)"
))

fig.add_vline(
    x=btc.index[-1],
    line_dash="dot",
    opacity=0.6
)

fig.update_layout(
    title="Bitcoin Volatility: Last 7 Days vs 5-Day Forecast",
    xaxis_title="Date",
    yaxis_title="5-Day Realized Volatility",
    template="plotly_dark",
    hovermode="x unified",
    legend_title_text="",
    margin=dict(l=40, r=20, t=60, b=40),
    font=dict(family="Inter, system-ui, -apple-system, Segoe UI, Roboto, Arial", size=14)
)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(tickformat=".4f", gridcolor="rgba(255,255,255,0.08)")

fig.show()

next_vol_pred

0.024626262485980988

In [113]:
import joblib
joblib.dump(best_model, "models/btc_vol_xgb.joblib")

['models/btc_vol_xgb.joblib']